In [ ]:
from modelvshuman_dmc import constants as c
from modelvshuman_dmc.plotting.plot import a, get_dataset_names, get_experiments, get_human_and_CNN_subjects
from modelvshuman_dmc.helper import plotting_helper as ph
from examples.simclr_hn_models import plotting_definitions

In [ ]:
plotting_def_name = 'plotting_definition_alexnets_simclr_hn'
plotting_definition = plotting_definitions.__dict__[plotting_def_name]
plotting_definition

In [ ]:
# plotting_definition()

In [ ]:
plot_types = c.DEFAULT_PLOT_TYPES
plot_types = ['accuracy']
plot_types

In [ ]:
# Plot(plot_types = plot_types, plotting_definition = plotting_def, figure_directory_name = figure_dirname)

In [ ]:
extra = ['cue-conflict', 'edge', 'silhouette', 'sketch', 'stylized']

In [ ]:
plot_type = 'accuracy'
current_dataset_names = get_dataset_names(plot_type)
print(current_dataset_names)
datasets = get_experiments(current_dataset_names)
len(datasets)

In [ ]:
analysis = a.SixteenClassAccuracy()
analysis

In [ ]:
decision_maker_fun = plotting_definition
decision_maker_fun

In [ ]:
# def plot_accuracy(datasets, decision_maker_fun, result_dir):
#     plot_general_analyses(datasets=datasets, analysis=a.SixteenClassAccuracy(),
#                           decision_maker_fun=decision_maker_fun,
#                           result_dir=result_dir, plot_type="accuracy")

In [ ]:
len(datasets)

In [ ]:
for idx,d in enumerate(datasets):
    print(idx, d.name)

In [ ]:
for idx,d in enumerate(datasets):

    df = ph.get_experimental_data(d)

    for e in d.experiments:
        decision_makers = decision_maker_fun(df)
        result_df = analysis.get_result_df(df=df,
                                           decision_makers=decision_makers,
                                           experiment=e)
        break
    
    if idx==11:
        print(d)
        break
        # x_y_plot(figure_path=figure_path,
        #          result_df=result_df,
        #          decision_makers=decision_makers,
        #          analysis=analysis,
        #          experiment=e)
    

In [ ]:
df

In [ ]:
len(df[df.subj=='alexnet_anime_sgd_lr005'])
cond1 = df[(df.subj=='alexnet_anime_sgd_lr005') & (df.condition=='cr')].sort_values(by="filename")
cond2 = df[(df.subj=='alexnet_anime_sgd_lr005') & (df.condition=='bw')].sort_values(by="filename")
len(cond1), len(cond2)

In [ ]:
subjects = df.subj.unique()
humans, dnns = get_human_and_CNN_subjects(subjects.tolist())
print(humans)
print(dnns)

In [ ]:
from scipy.stats import pearsonr

conditions = df.condition.unique()
cond = conditions[0]

subj1 = 'alexnet_anime_sgd_lr005'
subj2 = 'alexnet_lgn2_w1_mlp_simclrhn_probe1'
# subj2 = 'clip'
subset1 = df[(df.subj==subj1) & (df.condition==cond)].sort_values(by="filename")
subset2 = df[(df.subj==subj2) & (df.condition==cond)].sort_values(by="filename")
assert (subset1.filename==subset2.filename).all()
values1 = subset1.decision_margin.values.squeeze()
values2 = subset2.decision_margin.values.squeeze()
pearsonr(values1, values2)

In [ ]:
import seaborn as sns

ax = sns.scatterplot(x=values1, y=values2)
ax.axis('square')

In [ ]:
from modelvshuman_dmc.helper.dmc import compute_consistency

In [ ]:
compute_consistency(subset1, subset2)

# analyze human data

We're using the average percent correct across people to estimate the per-item decision margin. We need error bars / ceiling on the human data. We'll get that by correlating percent correct across halves, then using spearman brown to estimate the reliability of the full data set.

In [ ]:
import itertools
import torch
import numpy as np
from scipy.stats import pearsonr, spearmanr

def spearman_brown(r):
    '''prophecy formula (Brown, 1910; Spearman, 1910)'''
    return (2*r)/(1+r)

def get_split_halves(n):
    subjects = set(range(n))
    all_combos = list(itertools.combinations(subjects, n//2))
    all_splits = [[tuple(combo), tuple(subjects - set(combo))] for combo in all_combos]
    
    # make sure we don't repeat splits (reliability calc is symmetric)
    chunks = []
    splits = []
    for a,b in all_splits:
        if a in chunks: continue
        chunks.append(a)
        chunks.append(b)
        splits.append([a,b])
    
    for snum, split in enumerate(splits):    
        #print(split)
        assert subjects == set(split[0]+split[1]), f'Oops, split missing subjects {snum}'
        
    return splits

In [ ]:
from fastprogress import progress_bar

def compute_reliability_from_halves(df):
    subjects = df.subj.unique()
    results = defaultdict(list)
    N = len(subjects)
    halves = get_split_halves(N)
    for split_num,(splitA,splitB) in enumerate(progress_bar(halves)):
        subsA = np.array(subjects)[np.array(splitA)]
        subsB = np.array(subjects)[np.array(splitB)]

        subsetA = df[df.subj.isin(subsA)]
        subsetB = df[df.subj.isin(subsB)]

        avgA = subsetA.groupby(by=['filename']).mean(numeric_only=True).reset_index()
        avgB = subsetB.groupby(by=['filename']).mean(numeric_only=True).reset_index()
        assert (avgA.filename == avgB.filename).all()
        corr = pearsonr(avgA.is_correct, avgB.is_correct)[0]

        results['split_num'].append(split_num)
        results['splitA'].append(splitA)
        results['splitB'].append(splitB)
        results['pearsonr'].append(corr)
        
    human_split_corr = pd.DataFrame(results)
    mean_corr = human_split_corr.pearsonr.mean()
    reliability = spearman_brown(mean_corr)
    
    return human_split_corr, reliability

In [ ]:
human_df = df[df.subj.isin(humans)].copy().reset_index()
human_df.loc[:,'filename'] = human_df.imagename.apply(lambda x: "_".join(x.split("_")[-2:]))
human_df.loc[:,'is_correct'] = (human_df.object_response==human_df.category).astype(float)
human_df

In [ ]:
human_df = human_df.groupby(by=['filename']).mean(numeric_only=True).reset_index()
human_df.insert(0, 'subj', 'human_avg')
human_df = human_df.drop(columns=['index', 'session', 'trial', 'targ_act', 'max_nontarg_act'])
human_df.loc[:,'decision_margin'] = human_df.loc[:,'is_correct']
human_df = human_df.sort_values(by="filename")
human_df

In [ ]:
dnn_df = df[df.subj.isin(dnns)]
dnn_df

In [ ]:
import pandas as pd

df_ = pd.concat([dnn_df, human_df])
df_

In [ ]:
from collections import defaultdict

results = defaultdict(list)
subset1 = human_df
values1 = subset1.decision_margin.values.squeeze()
for dnn in dnns:
    subset2 = df[df.subj==dnn].sort_values(by="filename")
    assert (subset1.filename.values==subset2.filename.values).all()    
    values2 = subset2.decision_margin.values.squeeze()
    corr = pearsonr(values1, values2)[0]
    results['dnn'].append(dnn)
    results['decision_margin_consistency'].append(corr)
results_df = pd.DataFrame(results)
results_df = results_df.sort_values(by="decision_margin_consistency", ascending=False)
results_df

In [ ]:
%config InlineBackend.figure_format='retina'

In [ ]:
modelvsmodel_df = compute_consistency(dnn_df)
modelvsmodel_df.decision_margin_consistency.mean()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Adjust figure size
plt.figure(figsize=(10, 6))  # Width: 10, Height: 6

ax = sns.barplot(data=results_df, x="dnn", y="decision_margin_consistency")
plt.xticks(rotation=90);

# redux, draft analysis

extra = ['cue-conflict', 'edge', 'silhouette', 'sketch', 'stylized']


In [ ]:
import os
from pathlib import Path
from glob import glob
from natsort import natsorted

root_dir = '../outputs/raw-data/'
experiments = natsorted([Path(f.path).name for f in os.scandir(root_dir) if f.is_dir()])
experiments

In [ ]:
experiment = 'edge'
experiment = 'sketch'
experiment = 'silhouette'
experiment = 'stylized'
experiment = 'cue-conflict'
experiment = 'colour'
files = glob(os.path.join(root_dir, experiment, '*.csv'))
human_files = natsorted([filename for filename in files if "_subject-" in filename])
dnn_files = natsorted([filename for filename in files if "_subject-" not in filename])
dnn_df = pd.concat([pd.read_csv(filename) for filename in dnn_files])
# dnn_df.loc[:,'condition'] = 0
# dnn_df['condition'] = dnn_df['condition'].astype(int)
dnn_df

In [ ]:
human_df = pd.concat([pd.read_csv(filename) for filename in human_files])
# human_df.loc[:,'filename'] = human_df.imagename.apply(lambda x: "_".join(x.split("_")[-1:]))
human_df.loc[:,'filename'] = human_df.imagename.apply(lambda x: "_".join(x.split("_")[-2:]))
human_df.loc[:,'is_correct'] = (human_df.object_response==human_df.category).astype(float)
human_df.loc[:,'decision_margin'] = (human_df.object_response==human_df.category).astype(float)
human_df

In [ ]:
modelvsmodel = compute_consistency(dnn_df)
modelvsmodel

In [ ]:
modelvsmodel.error_consistency.mean()

In [ ]:
modelvsmodel.decision_margin_consistency.mean()

In [ ]:
ax = sns.scatterplot(x=modelvsmodel.error_consistency,
                     y=modelvsmodel.decision_margin_consistency)
ax.axis('square')

In [ ]:
human_split_corr, human_reliability = compute_reliability_from_halves(human_df)
human_reliability

In [ ]:
human_split_corr

In [ ]:
# human_df[human_df.subj=='subject-01']

In [ ]:
avg_human_df = human_df.groupby(by=['filename']).mean(numeric_only=True).reset_index()
avg_human_df.insert(0, 'subj', 'human_avg')
if 'session' in avg_human_df.columns:
    avg_human_df = avg_human_df.drop(columns=['session', 'trial'])
else:
    avg_human_df = avg_human_df.drop(columns=['trial'])
avg_human_df.loc[:,'decision_margin'] = avg_human_df.loc[:,'is_correct']
avg_human_df = avg_human_df.sort_values(by="filename")
avg_human_df

In [ ]:
from collections import defaultdict

results = defaultdict(list)
subset1 = avg_human_df
values1 = avg_human_df.is_correct.values.squeeze()
for dnn in dnn_df.subj.unique():
    subset2 = dnn_df[dnn_df.subj==dnn].sort_values(by="filename")
    assert (subset1.filename.values==subset2.filename.values).all()    
    values2 = subset2.decision_margin.values.squeeze()
    corr = pearsonr(values1, values2)[0]
    results['dnn'].append(dnn)
    results['decision_margin_consistency'].append(corr)
results_df = pd.DataFrame(results)
results_df = results_df.sort_values(by="decision_margin_consistency", ascending=False)
results_df

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Adjust figure size
plt.figure(figsize=(10, 6))  # Width: 10, Height: 6

ax = sns.barplot(data=results_df, x="dnn", y="decision_margin_consistency")
ax.axhline(y=human_reliability, color='gray', linestyle='-', linewidth=1.5)

plt.xticks(rotation=90);

In [ ]:
modelvshuman = compute_consistency(dnn_df, human_df)
modelvshuman

In [ ]:
modelvshuman_avg = modelvshuman.groupby(by=['sub1']).mean(numeric_only=True).reset_index()
modelvshuman_avg.head()

In [ ]:
# Adjust figure size
plt.figure(figsize=(10, 6))  # Width: 10, Height: 6
ax = sns.barplot(data=modelvshuman_avg, x="sub1", y="error_consistency")
ax.set_ylim([0, .9])
plt.xticks(rotation=90);

In [ ]:
# Adjust figure size
plt.figure(figsize=(10, 6))  # Width: 10, Height: 6

ax = sns.barplot(data=modelvshuman_avg, x="sub1", y="decision_margin_consistency")
ax.axhline(y=human_reliability, color='gray', linestyle='-', linewidth=1.5)
ax.set_ylim([0, .9])

plt.xticks(rotation=90);